# Transforming and joining raw data

The "raw" data is divided among the following tables:

- **Customer metadata**
  - customerID
  - gender
  - date of birth (we'll derive age and senior citizen status from this)
  - Partner
  - Dependents
  - (nominal) MonthlyCharges
- **Billing events**
  - customerID
  - date (we'll derive tenure from the number/duration of billing events)
  - kind (one of "AccountCreation", "Charge", or "AccountTermination")
  - value (either a positive nonzero amount or 0.00; we'll derive TotalCharges from the sum of amounts and Churn from the existence of an AccountTermination event)
- **Customer phone features**
  - customerID
  - feature (one of "PhoneService" or "MultipleLines")
- **Customer internet features**
  - customerID
  - feature (one of "InternetService", "OnlineSecurity", "OnlineBackup", "DeviceProtection", "TechSupport", "StreamingTV", "StreamingMovies")
  - value (one of "Fiber", "DSL", "Yes", "No")
- **Customer account features**
  - customerID
  - feature (one of "Contract", "PaperlessBilling", "PaymentMethod")
  - value (one of "Month-to-month", "One year", "Two year", "No", "Yes", "Credit card (automatic)", "Mailed check", "Bank transfer (automatic)", "Electronic check")

We want to join these together to reconstitute a training data set with this schema:

- customerID
- gender
- SeniorCitizen
- Partner
- Dependents
- tenure
- PhoneService
- MultipleLines
- InternetService
- OnlineSecurity
- OnlineBackup
- DeviceProtection
- TechSupport
- StreamingTV
- StreamingMovies
- Contract
- PaperlessBilling
- PaymentMethod
- MonthlyCharges
- TotalCharges
- Churn

In [1]:
# notebook parameters

import os

os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3.9"
os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/bin/python3.9"
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-11.0.12.0.7-4.fc34.x86_64"

spark_master = "local[*]"
app_name = "churn-etl"
input_files = dict(
    billing="billing_events", 
    account_features="customer_account_features", 
    internet_features="customer_internet_features", 
    meta="customer_meta", 
    phone_features="customer_phone_features"
)
output_file = "churn-etl"
output_prefix = ""
output_mode = "overwrite"
output_kind = "parquet"
input_kind = "parquet"
driver_memory = '8g'
executor_memory = '8g'


In [2]:
import pyspark

session = pyspark.sql.SparkSession.builder \
    .master(spark_master) \
    .appName(app_name) \
    .config("spark.eventLog.enabled", True) \
    .config("spark.eventLog.dir", ".") \
    .config("spark.driver.memory", driver_memory) \
    .config("spark.executor.memory", executor_memory) \
    .config("spark.executor.cores", 1) \
    .config("spark.rapids.sql.concurrentGpuTasks", 1) \
    .config("spark.rapids.memory.pinnedPool.size", "2G") \
    .config("spark.locality.wait", "0s") \
    .config("spark.sql.files.maxPartitionBytes", "512m") \
    .config("spark.plugins", "com.nvidia.spark.SQLPlugin") \
    .config("spark.jars", "/opt/sparkRapidsPlugin/cudf-21.08.2-cuda11.jar,/opt/sparkRapidsPlugin/rapids-4-spark_2.12-21.08.0.jar") \
    .getOrCreate()
session

21/09/25 14:47:07 WARN Utils: Your hostname, virt resolves to a loopback address: 127.0.0.1; using 192.168.86.109 instead (on interface wlp2s0)
21/09/25 14:47:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/09/25 14:47:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/09/25 14:47:21 WARN GpuDeviceManager: Initial RMM allocation (3437.32421875 MB) is larger than the adjusted maximum allocation (3018.375 MB), lowering initial allocation to the adjusted maximum allocation.
21/09/25 14:47:23 WARN SQLExecPlugin: RAPIDS Accelerator 21.08.0 using cudf 21.08.2. To disable GPU support set `spark.rapids.sql.enabled` to false
21/09/25 14:47:23 WARN Plugin: Installing rapids UDF compi

In [3]:
import churn.etl

churn.etl.register_options(
    spark_master = spark_master,
    app_name = app_name,
    input_files = input_files,
    output_prefix = output_prefix,
    output_mode = output_mode,
    output_kind = output_kind,
    input_kind = input_kind,
    driver_memory = driver_memory,
    executor_memory = executor_memory
)

# Reconstructing billing events and charges

In [4]:
from churn.etl import read_df
billing_events = read_df(session, input_files["billing"])
billing_events.printSchema()

root
 |-- customerID: string (nullable = true)
 |-- kind: string (nullable = true)
 |-- value: decimal(8,2) (nullable = true)
 |-- date: date (nullable = true)
 |-- month: string (nullable = true)



In [5]:
from churn.etl import join_billing_data
customer_billing = join_billing_data(billing_events)

In [6]:
customer_billing

DataFrame[customerID: string, Churn: boolean, tenure: bigint, TotalCharges: decimal(18,2)]

When we aggregated billing data, we also captured a unique list of customers in a temporary view.  For convenience, we can access it as follows:

In [7]:
from churn.etl import customers as get_customers
customers = get_customers()

# Reconstructing phone features


In [8]:
phone_features = read_df(session, input_files["phone_features"])
phone_features.printSchema()

root
 |-- customerID: string (nullable = true)
 |-- feature: string (nullable = true)
 |-- value: string (nullable = true)



In [9]:
from churn.etl import join_phone_features
customer_phone_features = join_phone_features(phone_features)

# Reconstructing internet features

Whereas phone features only include whether or not there are multiple lines, there are several internet-specific features in accounts:

- `InternetService` (one of `Fiber optic` or `DSL` in the "raw" data; its absence translates to `No` in the processed data)
- `OnlineSecurity` (`Yes` in the "raw" data if present; one of `No`, `Yes`, or `No internet service` in the processed data)
- `OnlineBackup` (`Yes` in the "raw" data if present; one of `No`, `Yes`, or `No internet service` in the processed data)
- `DeviceProtection` (`Yes` in the "raw" data if present; one of `No`, `Yes`, or `No internet service` in the processed data)
- `TechSupport` (`Yes` in the "raw" data if present; one of `No`, `Yes`, or `No internet service` in the processed data)
- `StreamingTV` (`Yes` in the "raw" data if present; one of `No`, `Yes`, or `No internet service` in the processed data)
- `StreamingMovies` (`Yes` in the "raw" data if present; one of `No`, `Yes`, or `No internet service` in the processed data)

This will lead to some slightly more interesting joins!

In [10]:
internet_features = read_df(session, input_files["internet_features"])
internet_features.printSchema()
internet_features.show()

root
 |-- customerID: string (nullable = true)
 |-- feature: string (nullable = true)
 |-- value: string (nullable = true)



+--------------------+---------------+-----+
|          customerID|        feature|value|
+--------------------+---------------+-----+
|7590-VHVEG-Mg8VG5...|InternetService|  DSL|
|7590-VHVEG-5xLi5Z...|InternetService|  DSL|
|7590-VHVEG-ZePlJi...|InternetService|  DSL|
|7590-VHVEG-x9IoNd...|InternetService|  DSL|
|7590-VHVEG-Z9yCIk...|InternetService|  DSL|
|7590-VHVEG-K8kBya...|InternetService|  DSL|
|7590-VHVEG-4ZjnIU...|InternetService|  DSL|
|7590-VHVEG-0stTDJ...|InternetService|  DSL|
|7590-VHVEG-lqhKlh...|InternetService|  DSL|
|7590-VHVEG-4Y_zUA...|InternetService|  DSL|
|7590-VHVEG-34V86Q...|InternetService|  DSL|
|7590-VHVEG-GCNzU2...|InternetService|  DSL|
|7590-VHVEG-i0AFUE...|InternetService|  DSL|
|7590-VHVEG-F1ALBc...|InternetService|  DSL|
|7590-VHVEG-aEfHl7...|InternetService|  DSL|
|7590-VHVEG-3K15yQ...|InternetService|  DSL|
|7590-VHVEG-eiqTDe...|InternetService|  DSL|
|7590-VHVEG-iMYyeZ...|InternetService|  DSL|
|7590-VHVEG-rReekB...|InternetService|  DSL|
|7590-VHVE

In [11]:
from churn.etl import join_internet_features
customer_internet_features = join_internet_features(internet_features)

# Reconstructing account features

In [12]:
account_features = read_df(session, input_files["account_features"])
account_features.printSchema()
account_features.show()

root
 |-- customerID: string (nullable = true)
 |-- feature: string (nullable = true)
 |-- value: string (nullable = true)

+--------------------+-------------+----------------+
|          customerID|      feature|           value|
+--------------------+-------------+----------------+
|7590-VHVEG-Mg8VG5...|PaymentMethod|Electronic check|
|7590-VHVEG-5xLi5Z...|PaymentMethod|Electronic check|
|7590-VHVEG-ZePlJi...|PaymentMethod|Electronic check|
|7590-VHVEG-x9IoNd...|PaymentMethod|Electronic check|
|7590-VHVEG-Z9yCIk...|PaymentMethod|Electronic check|
|7590-VHVEG-K8kBya...|PaymentMethod|Electronic check|
|7590-VHVEG-4ZjnIU...|PaymentMethod|Electronic check|
|7590-VHVEG-0stTDJ...|PaymentMethod|Electronic check|
|7590-VHVEG-lqhKlh...|PaymentMethod|Electronic check|
|7590-VHVEG-4Y_zUA...|PaymentMethod|Electronic check|
|7590-VHVEG-34V86Q...|PaymentMethod|Electronic check|
|7590-VHVEG-GCNzU2...|PaymentMethod|Electronic check|
|7590-VHVEG-i0AFUE...|PaymentMethod|Electronic check|
|7590-VHVEG-

In [13]:
from churn.etl import join_account_features
customer_account_features = join_account_features(account_features)

# Account metadata

In [14]:
account_meta = read_df(session, input_files["meta"])
account_meta.printSchema()

root
 |-- customerID: string (nullable = true)
 |-- dateOfBirth: date (nullable = true)
 |-- gender: string (nullable = true)
 |-- SeniorCitizen: string (nullable = true)
 |-- Partner: string (nullable = true)
 |-- Dependents: string (nullable = true)
 |-- MonthlyCharges: decimal(8,2) (nullable = true)
 |-- now: timestamp (nullable = true)



In [15]:
from churn.etl import process_account_meta
customer_account_meta = process_account_meta(account_meta)

# Putting it all together

In [16]:
from churn.etl import chained_join
from churn.etl import forcefloat

wide_data = chained_join(
    "customerID",
    customers,
    [
        customer_billing,
        customer_phone_features,
        customer_internet_features,
        customer_account_features,
        customer_account_meta
    ]
).select(
    "customerID", 
    "gender", 
    "SeniorCitizen", 
    "Partner", 
    "Dependents", 
    "tenure", 
    "PhoneService", 
    "MultipleLines", 
    "InternetService", 
    "OnlineSecurity", 
    "OnlineBackup", 
    "DeviceProtection", 
    "TechSupport", 
    "StreamingTV", 
    "StreamingMovies", 
    "Contract", 
    "PaperlessBilling", 
    "PaymentMethod", 
    forcefloat("MonthlyCharges"),
    forcefloat("TotalCharges"), 
    "Churn"
)

In [17]:
wide_data.explain()

== Physical Plan ==
*(10) Project [customerID#0, gender#237, SeniorCitizen#251, Partner#239, Dependents#240, tenure#60L, PhoneService#93, MultipleLines#97, InternetService#177, OnlineSecurity#178, OnlineBackup#179, DeviceProtection#180, TechSupport#181, StreamingTV#182, StreamingMovies#183, Contract#211, PaperlessBilling#230, PaymentMethod#217, cast(MonthlyCharges#241 as float) AS MonthlyCharges#338, cast(TotalCharges#61 as float) AS TotalCharges#339, Churn#36]
+- *(10) BroadcastHashJoin [customerID#0], [customerID#235], LeftOuter, BuildRight, false
   :- *(10) Project [customerID#0, Churn#36, tenure#60L, TotalCharges#61, PhoneService#93, MultipleLines#97, InternetService#177, OnlineSecurity#178, OnlineBackup#179, DeviceProtection#180, TechSupport#181, StreamingTV#182, StreamingMovies#183, Contract#211, PaperlessBilling#230, PaymentMethod#217]
   :  +- SortMergeJoin [customerID#0], [customerID#296], LeftOuter
   :     :- *(8) Project [customerID#0, Churn#36, tenure#60L, TotalCharges#61

In [18]:
%%time
from churn.etl import write_df
write_df(wide_data, output_file)

CPU times: user 810 ms, sys: 146 ms, total: 956 ms
Wall time: 3min 40s


# Inspecting individual tables

If we need to inspect individual components of our processing, we can.  Each constituent of these joins is registered as a temporary view.  For example, we loaded `customers` earlier using a method from `churn.etl`, but it is also available as a table:

In [19]:
customers = session.table("customers")

In [20]:
customers.show()

+--------------------+
|          customerID|
+--------------------+
|0082-OQIQY-pUoKfE...|
|5222-IMUKT-iQmvCv...|
|9507-EXLTT-3UzP9O...|
|7722-VJRQD-rt81Nn...|
|7294-TMAOP-F7eB8h...|
|8150-QUDFX-fW0@wc...|
|1268-ASBGA-OACqzJ...|
|2645-QTLMB-PjdWrt...|
|1855-CFULU-Mg8VG5...|
|5130-YPIRV-pCuX1v...|
|7665-VIGUD-Z9yCIk...|
|7426-RHZGU-uZXrFc...|
|0022-TCJCI-lqhKlh...|
|5575-TPIZQ-owpAU0...|
|7010-ZMVBF-AwBEdx...|
|3913-RDSJZ-owpAU0...|
|1875-QIVME-3K15yQ...|
|2397-BRLOM-OACqzJ...|
|6870-ZWMNX-bsGHcc...|
|2233-TXSIU-3tuH6B...|
+--------------------+
only showing top 20 rows



We can see which tables are available by querying the session catalog:

In [21]:
tables = session.catalog.listTables()
[t.name for t in tables]

['churned',
 'contracts',
 'counts_and_charges',
 'customer_account_features',
 'customer_account_meta',
 'customer_billing',
 'customer_charges',
 'customer_internet_features',
 'customer_phone_features',
 'customers',
 'device_protection',
 'internet_service',
 'multiple_lines',
 'online_backup',
 'online_security',
 'paperless',
 'payment',
 'phone_service',
 'streaming_movies',
 'streaming_tv',
 'tech_support',
 'terminations']

# Finishing up

In [ ]:
session.stop()